##[장르별 분석] 

- 발라드 가사를 문장 생성 모델 학습에 선정한 이유








---




##1. 장르별 특성을 추출하기 위한 단계
- 인디, 국악, 힙합, 발라드, 트로트, 록/메탈, 포크/블루스, 댄스, RnB 등의 장르 시각화 분석을 통한 insight 도출 시도

- 가사 길이 시각화, 워드 클라우드 활용 등으로 가시성 확보

- 통일성을 위해 같은 코드로 시각화

- 조원 분들의 eda 참고(지영님, 자현님, 종현님, 의준님, 지혜님) 및 통합


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly as pt
import seaborn as sns
import re

In [ ]:
# konlpy

!pip install konlpy --upgrade

In [ ]:
# 장르 데이터 파일 블러오기

data = pd.read_csv("")
data.head()

In [ ]:
# 중복 제거

data = data.drop_duplicates()

In [ ]:
data.shape

In [ ]:
data["가사"]

In [ ]:
data.info()

In [ ]:
df = data.copy()
df.head(1)

In [ ]:
# /n 제거 및 한글만 남김 => 문장 생성 처음에는 다른 언어가 섞일 시 제대로 된 문장이 생성되지 않을 수 있음
# 추후 영어 및 다른 언어 포함가능


for i in range(len(data)):
    df["가사"][i] = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", " ",df["가사"][i])
    # 숫자, 영어, 한글만 남기고 싶을 때: [^A-Za-z0-9가-힣]

df.head(1)


In [ ]:
df.sample(3)

In [ ]:
# 워드 클라우드 용 : lyrics


lyrics = df["가사"].values.tolist()
lyrics[:5]

##2. 가사길이
- 짧은 가사길이: 파편화된 구, 서사 파악이 어려울 수 있음
- 각 장르별 인기순위에 따른 음악의 가사 길이를 비교해보고 insight를 도출하고자 함
- 종현님 아이디어 👍

In [ ]:
# 가사 길이
# 한글로만 파악(문장 생성에 사용가능한 가사 길이 확인)

df["가사"].str.len().hist(bins=50).set_title("가사 길이")

In [ ]:
# 발라드: 공백 포함 약 300~600자 사이에 가장 많이 분포(종현님)
# 단, 오차 존재

##3. 워드클라우드
- 최빈 단어들을 모아 시각화 (한 자리 단어도 포함: 단, 상위 3개 단어 제외)
- 장르적 특성을 파악할 수 있는 주요한 단서가 될 수 있을 거라 생각

In [ ]:
# 워드클라우드

!apt-get update -qq
!apt-get install fonts-nanum* -qq  #코랩에서 한글 표기

In [ ]:
# konlpy 활용


from konlpy.tag import Kkma

kkma = Kkma()
kkma

In [ ]:
# 명사만 남겨서 워드클라우드 돌리기 

from konlpy.tag import Okt
okt = Okt()

In [ ]:
# range 안에 숫자 조정: 빈도를 파악하기 위해 인기순으로 1000곡만 가져와 장르별 분석 진행

y = []
for i in range(1000):
    nouns = okt.nouns(lyrics[i])
    y = y + nouns

y[:10]

In [ ]:
# lyrics 1000개 중
# 워드클라우드로 표현하기 위해 key(단어), value(빈도수)로 변환

count={}

for i in y:
    try: count[i] += 1
    except: count[i]=1

print(count)

In [ ]:
#  명사라고 여겨지는 것 추출 및 빈도수 별 시리즈 변환
 
y_freq = pd.Series(count)
a = y_freq.sort_values(ascending=False)
a


# b = pd.DataFrame(a[(1650 > a)&(a > 10)], columns=["빈도"]) => 인디 분석할 때 썼던 코드 상위 3개 단어 잘라내기 위한 것
# b ==> 데이터마다 조정해서 사용


In [ ]:
# 워드클라우드 생성


word = pd.DataFrame({"단어" : b.index.tolist(),
              "빈도수": b["빈도"]})

wordcloud = WordCloud(
            font_path="NanumGothic",
            width=800,
            height=800
)
text = " ".join(word["단어"])
keyword = wordcloud.generate(text)

array = keyword.to_array()

plt.figure(figsize=(10, 10))
plt.imshow(
    array,
    interpolation="bilinear"
)
plt.axis("off")
plt.show()

In [ ]:
# 워드 클라우드 생성에 사용한 text를 확인하여 각 장르별 특성 파악

text

##4. 기타 시각화
- 생각치 못한 인사이트를 발견할 수 있기를 바라며
- 정리를 위해 작성 (지영님, 종현님, 자현님 코드 참고 명시, 개별 EDA한 것을 자료로 활용하여 조원들의 노력이 프로젝트에 직접적으로 이어지도록 하기 위함)
- 지영님 코드 👍


In [ ]:
# 기타 시각화
# 지영님 코드 참고 명시

L = data["가사"]

# 가사 길이 추가
# 단어 개수 추가
data["가사"] = data["가사"].str.strip()
data["len"] = data["가사"].str.len()

data["word_count"] = L.map(lambda x : len(x.split()))
data["word_count"]

In [ ]:
data.hist(bins=55, figsize=(12, 7))

In [ ]:
# 영어 비율과 한글 비율 비교하기 위해 한글, 영어, 숫자를 제외한 문자 제거


for i in range(len(data)):
    data["가사"][i] = re.sub("[^A-Za-z0-9가-힣]", " ",data["가사"][i])

data.head()

In [ ]:

data["가사개행제거"] = data["가사"].map(lambda x: (" ".join(x.split("\r\n")).strip()))
data["가사개행제거"].sample(2)



In [ ]:
# 지영님 코드

english = re.compile(r'[a-zA-Z]')
korean = re.compile(r'[가-힣]')
number = re.compile(r'[0-9]')

def check_word_kind(dancd_lyrics):
    eng = 0
    kor = 0
    num = 0
    etc = 0
    
    for word in dancd_lyrics:
        if korean.match(word):
            kor += 1
        elif english.match(word):
            eng += 1
        elif word.isdigit():
            num += 1    
        else:
            etc += 1

    return [kor, eng, num, etc]


In [ ]:


data["가사의단어들"] = data["가사개행제거"].map(lambda x : x.split())
data["가사의단어들"].sample(1)



In [ ]:


temp = data["가사의단어들"].sample(1).tolist()
temp = temp[0]
temp



In [ ]:

check_word_kind(temp)


In [ ]:
def word_kind_count(df):
    kor_eng_num_etc = pd.DataFrame(df["가사의단어들"].map(check_word_kind))
    df["한글단어수"] = kor_eng_num_etc["가사의단어들"].str[0]
    df["영어단어수"] = kor_eng_num_etc["가사의단어들"].str[1]
    df["숫자단어수"] = kor_eng_num_etc["가사의단어들"].str[2]
    df["기타단어수"] = kor_eng_num_etc["가사의단어들"].str[3]

In [ ]:
word_kind_count(data)

In [ ]:
kor_eng_etc = pd.DataFrame(dance["가사의단어들"].map(check_word_kind))
kor_eng_etc

In [ ]:
# data["한글단어수"] = kor_eng_etc["가사의단어들"].str[0]
# data["영어단어수"] = kor_eng_etc["가사의단어들"].str[1]
# data["숫자단어수"] = kor_eng_etc["가사의단어들"].str[2]
# data["기타단어수"] = kor_eng_etc["가사의단어들"].str[3]


# data.hist(figsize=(16,12), bins=55, color='black')

In [ ]:
# seaborn이 시각적으로 나음


fig, axes = plt.subplots(1, 2, figsize=(16, 12))
sns.histplot(data=data, x="영어단어수", bins=20, ax=axes[0][0]).set_title("댄스"). #장르마다 바꿔주기
sns.boxenplot(data=data, y="영어단어수", ax=axes[1][0]).set_title("댄스")



In [ ]:
data[['한글단어수', '영어단어수', '기타단어수', '숫자단어수']].describe()

In [ ]:
# 한글이 아닌 단어의 비율

data["한글아닌단어비율"] = (data['영어단어수'] + data['기타단어수'] + data['숫자단어수']) / data["word_count"] * 100
data["한글아닌단어비율"][:10]



In [ ]:
sns.displot(data=data, x="한글아닌단어비율", kde=True)

In [ ]:
# 

temp_pos = okt.pos(data["가사개행제거"][0])
temp_pos



In [ ]:
def count_pos(Lyrics):
    noun = 0
    verb = 0
    adjective = 0 #형용사
    adverb = 0 #부사
    etc = 0

    for word in Lyrics:
        if word[1] == "Noun":
            noun += 1
        elif word[1] == "Verb":
            verb += 1
        elif word[1] == "Adjective":
            adjective += 1
        elif word[1] == "Adverb":
            adverb += 1
        else:
            etc += 1
        
    return noun, verb, adjective, adverb, etc

count_pos(temp_pos)



In [ ]:
# 상위 100개 노래 가사만 가져와서 분석

data_100 = data.head(100)
data_100.shape

In [ ]:
data_100["형태소"] = dance_100["가사개행제거"].map(okt.pos)
data_100["형태소"][:2]

In [ ]:
data_pos = data_100["형태소"].map(count_pos)
data_pos

In [ ]:
# noun, verb, adjective, adverb, etc
data_100["명사"] = data_pos.str[0]
data_100["동사"] = data_pos.str[1]
data_100["형용사"] = data_pos.str[2]
data_100["부사"] = data_pos.str[3]
data_100["기타형태소"] = data_pos.str[4]



data_100[['형태소', '명사', '동사', '형용사', '부사']].describe()

In [ ]:
# plot

data_100[['한글단어수', '형태소', '명사', '동사', '형용사', '부사']].plot(kind="kde")

##5. 정리
1) 종합
- 
- 
- 
- 

2) 결론
- 발라드 가사를 모델 학습 데이터로 선정하는 이유는 

    ㄱ. 가

    ㄴ. 나

    ㄷ. 다

    ㄹ. 라

    위와 같다.

-

**
추후 발라드 데이터를 활용하여 직접적인 

-권소희 작성